## Завдання 2.
Використовуючи датасет з файлу **communities.csv**, виконайте нормалізацію даних. Визначте найкращий метод нормалізації для даного набору даних. 


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

import sklearn
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from numpy import genfromtxt

# from sklearn.preprocessing import scale
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV

In [10]:
data = pd.read_csv('communities.csv')
# x = data.isnull().sum()
# sum(x)

# col_names = data.columns.values.tolist()
# for col in col_names:
#     print(col)

# data.describe()
# data.info()
numeric_cols = [cname for cname in data.columns if data[cname].dtype in ['int64', 'float64']]
y = data['1']
X = data[numeric_cols].copy()
X.drop(['1'], axis=1, inplace=True)
data.head()

,8,?,?.1,Lakewoodcity,1,0.19,0.33,0.02,0.9,0.12,...,0.12.2,0.26.1,0.2.1,0.06.3,0.04.2,0.9.1,0.5.2,0.32.2,0.14.3,0.2.2
0,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.0,?,0.67
1,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.0,?,0.43
2,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.0,?,0.12
3,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.0,?,0.03
4,6,?,?,SouthPasadenacity,1,0.02,0.28,0.06,0.54,1.00,...,0.01,0.58,0.10,?,?,?,?,0.0,?,0.14


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=20)
# print(f"Кількість елементів у категоріях для навчального датасету {np.bincount(y_train)}")
# print(f"Кількість елементів у категоріях для тестового датасету {np.bincount(y_test)}")

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [13]:
print(f"Score for train set {lr.score(X_train, y_train)}")
print(f"Score for test set {lr.score(X_test, y_test)}")

Score for train set 0.07216714950558345
Score for test set -0.09711966353780332


### L1

In [101]:
lasso1 = Lasso(alpha=0.01).fit(X_train, y_train)
print(f"Score for train set {lasso1.score(X_train, y_train)}")
print(f"Score for test set {lasso1.score(X_test, y_test)}")

Score for train set 0.01688859921196617
Score for test set -0.0017232900499106751


### L2

In [107]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=20).fit(X_train, y_train)
print(f"Score for train set {ridge.score(X_train, y_train)}")
print(f"Score for test set {ridge.score(X_test, y_test)}")
# ridge.predict(X_test)

Score for train set 0.02912070978516046
Score for test set -0.007931632982761228


### L1 + L2

In [103]:
from sklearn.linear_model import ElasticNet
eklastic = ElasticNet(alpha=0.01, l1_ratio=0.01).fit(X_train, y_train)
print(f"Score for train set {eklastic.score(X_train, y_train)}")
print(f"Score for test set {eklastic.score(X_test, y_test)}")

Score for train set 0.0313157047199486
Score for test set -0.009003022085424917
